# 1. Country Pages Globals and Metadata

Global data and helper functions


https://production-api.globalforestwatch.org/v1/query/?sql=SELECT polyname, year_data.year, sum(year_data.area_loss) as sum FROM data WHERE polyname = 'wdpa'  and iso = 'BRA' GROUP BY polyname, iso, nested(year_data.year)


In [1]:
import requests
import requests_cache
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
requests_cache.install_cache('demo_cache')

Testing basic queries

**notes**

* New table structure includes `bound1-4` which provide an additional attribute to group by

```
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT polyname, count(*) FROM data WHERE iso = 'BRA' and thresh = 30 GROUP BY polyname
```

This one table should be enough to generate data for areas of interest from admin0 to admin 2 level on its own.

In [2]:
ds = '499682b1-3174-493f-ba1a-368b4636708e'  # ADMIN 2 level data

In [3]:
# Example of what is in the data tables and how to simply read them:
url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
#sql = (f"SELECT sum(area_gain) FROM {ds} WHERE iso = 'BRA' and adm1 = 7 AND thresh = 0 AND polyname = 'gadm28'")
#sql = (f"SELECT polyname, count(*) FROM {ds} WHERE iso = 'BRA' and thresh = 30 GROUP BY polyname")
sql = (f"SELECT * FROM {ds} WHERE iso = 'BRA' AND thresh = 30 AND polyname = 'plantations' LIMIT 2 ORDER BY area_extent DESC")
properties = {"sql": sql}
r = requests.get(url, params = properties)
print(r.url)
print(f'Status: {r.status_code}')
r.json()

https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+%2A+FROM+499682b1-3174-493f-ba1a-368b4636708e+WHERE+iso+%3D+%27BRA%27+AND+thresh+%3D+30+AND+polyname+%3D+%27plantations%27+LIMIT+2+ORDER+BY+area_extent+DESC
Status: 200


{'data': [{'_id': 'AWB6e_iNhRfkUZCTbvqD',
   'adm1': 11,
   'adm2': 1414,
   'area_extent': 57927.437055999995,
   'area_extent_2000': 39884.3891173,
   'area_gadm28': 1020790.7786600001,
   'area_gain': 67564.6094921,
   'area_poly_aoi': 151635.985063,
   'bound1': 'Large industrial plantation',
   'bound2': 'Wood fiber / timber',
   'bound3': -9999,
   'bound4': -9999,
   'iso': 'BRA',
   'polyname': 'plantations',
   'thresh': 30,
   'year_data': [{'area_loss': 450.900853092,
     'emissions': 36351.7897877,
     'year': 2001},
    {'area_loss': 1663.91938089, 'emissions': 196926.392146, 'year': 2002},
    {'area_loss': 1468.41710327, 'emissions': 211903.138267, 'year': 2003},
    {'area_loss': 4053.23034629, 'emissions': 539949.43733, 'year': 2004},
    {'area_loss': 2197.33341629,
     'emissions': 295112.97395300004,
     'year': 2005},
    {'area_loss': 1270.43151388, 'emissions': 204183.514617, 'year': 2006},
    {'area_loss': 718.068602042, 'emissions': 84105.1447234, 'year': 

The `polyname` attribute in the data indicates the geometry used in the intersection. E.g. `polyname`:`gadm28` means that the GADM28 polygons alone were used, therefore `area_extent` would be the value of the Hansen 2010 tree cover extent within a particular gadm28 geometry.

Charlie has used a convention of mutliple underscores to indicate when data has multiple intersects. The rule is as follows:
* All data are intersected with the gadm28
* Some data are intersected with gadam + something else, in that case it takes the name of the something else, e.g. plantations
* some data are intersected with multiple gadam + multiple other data sources, in that case, the data sources are seperated by a dunder (double underscore) in the `polyname`. E.g. `primary_forest__landmark` are the gadam28 + primary forest data + landmark 

In [4]:
url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
sql = (f"SELECT polyname FROM {ds} WHERE iso = 'BRA' and thresh = 0 GROUP BY polyname")
properties = {"sql": sql}
r = requests.get(url, params = properties)
print(r.url)
print(f'Status: {r.status_code}')
for row in r.json().get('data'):
    print(row.get('polyname'))

https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+polyname+FROM+499682b1-3174-493f-ba1a-368b4636708e+WHERE+iso+%3D+%27BRA%27+and+thresh+%3D+0+GROUP+BY+polyname
Status: 200
bra_biomes
gadm28
mining
plantations
plantations__mining
wdpa
primary_forest
primary_forest__mining
primary_forest__wdpa
plantations__wdpa
ifl_2013
ifl_2013__wdpa
ifl_2013__mining
landmark
primary_forest__landmark
plantations__landmark


Also of note, is the `bound1-4` attributes, which are 

```
   'bound1': 'Large industrial plantation',
   'bound2': 'Wood fiber / timber',
   'bound3': -9999,
   'bound4': -9999,
   
```

These hold extra properties of a dataset which can be used in group-by statements, e.g. plantation data has been split into species type. 

Finally, another key property is the `area_poly_aoi`, which should hold the total area (ha) of the shape used to make the intersection. E.g., the sum of the `polyname` = gadm28 `area_poly_aoi` values should equal the total land area of a selected ROI. 


<a id='s0'></a>
# Obtain metadata for admin-0 to admin-2 level

Retrieve json mappings of id values (e.g. iso codes or integers) for admin0 to admin2 level, and map them to english names for selectors and resources to build dynamic sentences.

In [5]:
def get_admin0_json(urlCarto="https://wri-01.carto.com/api/v2/sql"):
    """return an alphabetical json of mappings of iso and country names
    e.g.
    [{'iso': 'AFG', 'name': 'Afghanistan'},
     {'iso': 'XAD', 'name': 'Akrotiri and Dhekelia'},
     {'iso': 'ALA', 'name': 'Åland'},
     ...
    ]
    
    """
    sql = ("SELECT iso, country as name FROM umd_nat_staging "
            "GROUP BY iso, name "
            "ORDER BY name "
          )
    r = requests.get(urlCarto, params={"q": sql})
    return r.json().get('rows', None)
 

def get_admin1_json(iso, urlCarto="https://wri-01.carto.com/api/v2/sql"):
    """return an alphabetical json of mappings of admin1 for a country
    e.g. for iso='GBR'
        [{'adm1': 1, 'name': 'England'},
         {'adm1': 2, 'name': 'Northern Ireland'},
         {'adm1': 3, 'name': 'Scotland'},
         {'adm1': 4, 'name': 'Wales'}]
    """
    sql = ("SELECT id1 as adm1, region as name "
            "FROM umd_subnat_staging "
            f"WHERE iso = '{iso}' "
            "and year = 2001 "
            "and thresh = 30 "
            "ORDER BY name "
          )
    r = requests.get(urlCarto, params={"q": sql})
    return r.json().get('rows', None)
   

def get_admin2_json(iso, adm1, urlCarto="https://wri-01.carto.com/api/v2/sql"):
    """ 
    e.g. response for iso='GBR' adm1=1
    [{'adm2': 1, 'name': 'Barking and Dagenham'},
     {'adm2': 2, 'name': 'Bath and North East Somerset'},
     ...
     ]

    """
    sql = ("SELECT id_2 as adm2, name_2 as name "
            "FROM gadm28_adm2 "
            f"WHERE iso = '{iso}' "
            f"AND id_1 = {adm1} "
            "ORDER BY name "
          )
    r = requests.get(urlCarto, params={"q": sql})    
    return r.json().get('rows', None)

In [6]:
tmp = get_admin0_json()
iso_to_countries = {}
for row in tmp:
    iso_to_countries[row.get('iso')] = row.get('name')

# Get info on an areas shape

Use the Geostore service to find the geometry, bounding box or additional info on a given `iso` or `iso/admin1` area if required (e.g. for centering the map).

Note - if similar info is needed for admin2 areas I will have to go back and create a query based on a Carto table, as admin-2 level data does not exist in the geostore service yet.

In [7]:
iso = "GBR"
adm1 = None 

if not adm1:
    print("returning admin-0 level geostore")
    url = f"https://api.resourcewatch.org/v1/geostore/admin/{iso}"
else:
    print("returning admin-1 level geostore")
    url = f"https://api.resourcewatch.org/v1/geostore/admin/{iso}/{adm1}"
    
r = requests.get(url)
print(r.url)
roi = r.json()

print(roi.get('data', None).get('attributes', None).keys())
roi.get('data').get('attributes').get('bbox')  #e.g. of getting the bounding box of a ROI

returning admin-0 level geostore
https://api.resourcewatch.org/v1/geostore/admin/GBR
dict_keys(['geojson', 'hash', 'provider', 'areaHa', 'bbox', 'lock', 'info'])


[-13.6913900375365, 49.8654174804688, 1.76416802406322, 60.8458213806153]

In [8]:
def create_bounding_poly(bbox):
    """Create a bounding polygon from a returned bbox of the geostore service
        (- the old api produced shapes in this format).
    """
    lat1, lon1, lat2, lon2  = bbox
    return [[lat1,lon1], [lat1,lon2],[lat2,lon2],[lat2,lon1] ,[lat1,lon1]]

In [9]:
create_bounding_poly(roi.get('data').get('attributes').get('bbox'))

[[-13.6913900375365, 49.8654174804688],
 [-13.6913900375365, 60.8458213806153],
 [1.76416802406322, 60.8458213806153],
 [1.76416802406322, 49.8654174804688],
 [-13.6913900375365, 49.8654174804688]]

# Set up global (scope) data sources for the Country Widgets 

There is some work that needs to be done in populating the Location menu, not all options should be always available: some of them will need to be iso dependent. E.g. `Biomes` should only appear in the menu if the ISO is set to `BRA`


In [10]:
polynames = {
                'All Region':'gadm28',
                'Biomes':'bra_biomes',
                'Mining':'mining',
                'Protected Areas':'wdpa',
                'Primary Forests':'primary_forest',
                'Mining in Primary Forests':'primary_forest__mining',
                'Protected Areas in Primary Forests':'primary_forest__wdpa',
                'Intact Forest Landscapes':'ifl_2013',
                'Protected Areas in Intact Forest Landscapes':'ifl_2013__wdpa',
                'Mining in Intact Forest Landscapes':'ifl_2013__mining',
                'Indigenous Lands':'landmark',
                'Indigenous Lands in Primary Forests':'primary_forest__landmark',
                'Plantations':'plantations',
                'Mining in Plantation Areas':'plantations__mining',
                'Protected areas in Plantations':'plantations__wdpa',
                'Indigenous Lands in Plantations':'plantations__landmark',
            }

plantation_type_dict = {
    
                         'Large industrial plantation': 'large industrial',
                         'Clearing/ very young plantation': 'clearing or very young',
                         'Mosaic of medium-sized plantations': 'medium-sized',
                         'Mosaic of small-sized plantations': 'small sized'

                        }

plantation_species_dict = {
                            'Wood fiber / timber': 'wood fiber and timber',
                            'Wood fiber / timber mix': 'mixed wood fiber and timber',
                            'Recently cleared': 'recently cleared',
                            'Unknown': 'unknown',
                            'Fruit': 'fruit tree',
                            'Fruit mix': 'mixed fruit tree',
                            'Oil palm': 'palm tree',
                            'Other mix': 'mixed',
                            'Other': 'other',
                            'Rubber': 'rubber tree'
                            }

extent_year_dict = {
    
         2000:'area_extent_2000',
         2010:'area_extent'

        }

tabs = {
         "summary":"Summary",
         "land_cover":"Land Cover",
         "land_use": "Land Use",
         "forest_change": "Forest Change",
         "conservation":'Conservation',
         "people":"People",
         "climate":"Climate"
        }

# Dynamic Whitelists Tests

In [174]:
iso = "SAU"
adm1 = None
adm2 = None

In [175]:
def get_whitelist(iso, adm1=None, adm2=None):
    if adm2:
        print('Request for adm2 area')
        sql = (f"SELECT polyname FROM {ds} "
               "WHERE thresh = 0 AND polyname is not 'gadm28' "
               f"AND iso = '{iso}' AND adm1={adm1} AND adm2={adm2} "
               "GROUP BY polyname")
    elif adm1:
        print('Request for adm1 area')
        sql = (f"SELECT polyname FROM {ds} "
               "WHERE thresh = 0 AND polyname is not 'gadm28' "
               f"AND iso = '{iso}' AND adm1={adm1} "
               "GROUP BY polyname")
    else:
        print('Request for adm0 area')
        sql = (f"SELECT polyname FROM {ds} "
               "WHERE thresh = 0 AND polyname is not 'gadm28' "
               f"AND iso = '{iso}' "
               "GROUP BY polyname")
    return sql
    

In [176]:
%%time

# Create whitelist for each polyname

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
sql = get_whitelist(iso=iso, adm1=adm1, adm2=adm2)

properties = {"sql": sql}
r = requests.get(url, params = properties)
print(r.url)
print(f'Status: {r.status_code}')
data = r.json().get('data')
pprint(data)

Request for adm0 area
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+polyname+FROM+499682b1-3174-493f-ba1a-368b4636708e+WHERE+thresh+%3D+0+AND+polyname+is+not+%27gadm28%27+AND+iso+%3D+%27SAU%27+GROUP+BY+polyname
Status: 200
[{'polyname': 'wdpa'}]
CPU times: user 3.62 ms, sys: 2.48 ms, total: 6.1 ms
Wall time: 13.8 ms


In [177]:
unique_polys = []

for d in data:
    if d.get('polyname') not in unique_polys:
        unique_polys.append(d.get('polyname'))

        
unique_polys

['wdpa']

In [178]:
len(unique_polys)

1

###  Alternate Whitelist

This query can be used to assess whether polynames contain non-zero data.

There are cases where widgets do not need to be shown in cases where the data is zero (e.g. Saudi Arabia has Protected Areas, but there is no loss or gain for those areas - hence it is redundant to show any change)

This query returns the polynames that are found in the AOI ('gadm28, 'wdpa', 'ifl2013'...) as well as sum total of thier extent, loss and gain data. If a polyname is not present in teh AOI it will not be returned.

The returned object will have elements in following format:

```JSON

[{'polyname': value,
  'total_extent_2000': value,
  'total_extent_2010': value,
  'total_gain': value,
  'total_loss': value,
  },
  {...}]

```
In cases where there is zero total extent for the AOI, that key:value pair WILL NOT BE PRESENT:

e.g.

```JSON

[{'polyname': 'wdpa',
  'total_extent_2000': value,
  'total_extent_2010': value,
  'total_loss': value,
  },
  {...}]

```

e.g. in the above example, the protected areas in the AOI have had no gain.

In [184]:
iso = "SAU"
adm1 = None
adm2 = None

In [189]:
def get_alt_whitelist(iso, adm1=None, adm2=None):
    if adm2:
        print('Request for adm2 area')
        sql = ("SELECT polyname, SUM(area_extent_2000) as total_extent_2000, "
               "SUM(area_extent) as total_extent_2010, "
               "SUM(area_gain) as total_gain, "
               f"SUM(year_data.area_loss) as total_loss FROM {ds} "
               "WHERE thresh = 0 "
               f"AND iso = '{iso}' AND adm1={adm1} AND adm2={adm2} "
               "GROUP BY polyname")
    elif adm1:
        print('Request for adm1 area')
        sql = ("SELECT polyname, SUM(area_extent_2000) as total_extent_2000, "
               "SUM(area_extent) as total_extent_2010, "
               "SUM(area_gain) as total_gain, "
               f"SUM(year_data.area_loss) as total_loss FROM {ds} "
               "WHERE thresh = 0 "
               f"AND iso = '{iso}' AND adm1={adm1} "
               "GROUP BY polyname")
    else:
        print('Request for adm0 area')
        sql = ("SELECT polyname, SUM(area_extent_2000) as total_extent_2000, "
               "SUM(area_extent) as total_extent_2010, "
               "SUM(area_gain) as total_gain, "
               f"SUM(year_data.area_loss) as total_loss FROM {ds} "
               "WHERE thresh = 0 "
               f"AND iso = '{iso}' "
               "GROUP BY polyname")
    return sql
    

In [190]:
# Create whitelist for each polyname

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
sql = get_alt_whitelist(iso=iso, adm1=adm1, adm2=adm2)

properties = {"sql": sql}
r = requests.get(url, params = properties)
print(r.url)
print(f'Status: {r.status_code}')
data = r.json().get('data')
pprint(data)

Request for adm0 area
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+polyname%2C+SUM%28area_extent_2000%29+as+total_extent_2000%2C+SUM%28area_extent%29+as+total_extent_2010%2C+SUM%28area_gain%29+as+total_gain%2C+SUM%28year_data.area_loss%29+as+total_loss+FROM+499682b1-3174-493f-ba1a-368b4636708e+WHERE+thresh+%3D+0+AND+iso+%3D+%27SAU%27+GROUP+BY+polyname
Status: 200
[{'polyname': 'gadm28',
  'total_extent_2000': 21931.320770263672,
  'total_extent_2010': 26178.755500793457,
  'total_loss': 47},
 {'polyname': 'wdpa',
  'total_extent_2000': 328.40140199661255,
  'total_extent_2010': 448.2100884318352}]


In [191]:
unique_polys_alt = []

for d in data:
    if d.get('polyname') not in unique_polys_alt:
        unique_polys_alt.append(d.get('polyname'))

        
unique_polys_alt

['gadm28', 'wdpa']